In [29]:
import pandas as pd
import numpy as np
df = pd.read_csv("AWCustomers.csv")
print("shape",df.shape)
df.head()

shape (18361, 24)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06


In [14]:
selected_features = ["Education","Occupation","Gender","MaritalStatus","NumberCarsOwned","NumberChildrenAtHome","TotalChildren","YearlyIncome"]
df_selected = df[selected_features].copy()
df_selected.head()

,Education,Occupation,Gender,MaritalStatus,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,Bachelors,Clerical,M,M,3,0,1,81916
1,Partial College,Clerical,M,M,2,1,2,81076
2,Bachelors,Clerical,F,S,3,0,0,86387
3,Partial College,Skilled Manual,M,M,2,1,2,61481
4,Partial College,Skilled Manual,M,S,1,0,0,51804


In [13]:
from sklearn.preprocessing import MinMaxScaler

Scaler = MinMaxScaler()
df_selected.loc[:,"YearlyIncome"] = Scaler.fit_transform(df_selected[["YearlyIncome"]])
df_selected.head()

,Education,Occupation,Gender,MaritalStatus,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,Bachelors,Clerical,M,M,3,0,1,0.496842
1,Partial College,Clerical,M,M,2,1,2,0.489453
2,Bachelors,Clerical,F,S,3,0,0,0.536172
3,Partial College,Skilled Manual,M,M,2,1,2,0.317083
4,Partial College,Skilled Manual,M,S,1,0,0,0.231958


In [15]:
print("Yearly Income Stats:")
print(df_selected['YearlyIncome'].describe())

Yearly Income Stats:
count     18361.000000
mean      72754.779642
std       30686.014313
min       25435.000000
25%       53312.000000
50%       61851.000000
75%       87410.000000
max      139115.000000
Name: YearlyIncome, dtype: float64


In [19]:
bins = [0,50000,80000,df_selected["YearlyIncome"].max()]
labels = ["low","medium","high"]
df_selected["IncomeCategory"] = pd.cut(df_selected["YearlyIncome"],bins=bins,labels=labels,include_lowest = True)

df_selected[["YearlyIncome","IncomeCategory"]].head(10)

,YearlyIncome,IncomeCategory
0,81916,high
1,81076,high
2,86387,high
3,61481,medium
4,51804,medium
5,61944,medium
6,34919,low
7,61832,medium
8,83834,high
9,26880,low


In [23]:
df_encoded = pd.get_dummies(df_selected , columns = ["Education","Occupation","Gender","MaritalStatus","IncomeCategory"])
df_encoded = df_encoded.astype(int)
df_encoded.head()

,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Education_Bachelors,Education_Graduate Degree,Education_High School,Education_Partial College,Education_Partial High School,Occupation_Clerical,...,Occupation_Manual,Occupation_Professional,Occupation_Skilled Manual,Gender_F,Gender_M,MaritalStatus_M,MaritalStatus_S,IncomeCategory_low,IncomeCategory_medium,IncomeCategory_high
0,3,0,1,81916,1,0,0,0,0,1,...,0,0,0,0,1,1,0,0,0,1
1,2,1,2,81076,0,0,0,1,0,1,...,0,0,0,0,1,1,0,0,0,1
2,3,0,0,86387,1,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,1
3,2,1,2,61481,0,0,0,1,0,0,...,0,0,1,0,1,1,0,0,1,0
4,1,0,0,51804,0,0,0,1,0,0,...,0,0,1,0,1,0,1,0,1,0


In [30]:
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity

row1 = df_encoded.iloc[1]
row2 = df_encoded.iloc[2]
numeric_cols = ["NumberCarsOwned","NumberChildrenAtHome","TotalChildren","YearlyIncome"]
cos_sim = cosine_similarity([row1[numeric_cols]],[row2[numeric_cols]])

binary_cols = [col for col in df_encoded.columns if col not in numeric_cols]
jaccard = jaccard_score(row1[binary_cols], row2[binary_cols])
simple_match = np.mean(row1[binary_cols] == row2[binary_cols])

print("Cosine similarity:", cos_sim[0][0])
print("Jaccard similarity:", jaccard)
print("Simple Matching similarity:", simple_match)

Cosine similarity: 0.9999999995690805
Jaccard similarity: 0.25
Simple Matching similarity: 0.6470588235294118


In [31]:
correlation = df_encoded['NumberCarsOwned'].corr(df_encoded['YearlyIncome'])
print("Correlation:", correlation)

Correlation: 0.47730015236316986
